In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os
import re


<h2>Read the Data</h2>

Data will be provided to qualified researchers in a csv format.

In [ ]:
df = pd.read_csv('data.csv', delimiter='\t', index_col=0 )


# set the index of the sample date, etc.
df.set_index(['Tube_Label', 'Site','Collection_Date'], inplace=True)
# reshape the columns for easier indexing
df.columns = pd.MultiIndex.from_tuples([(int(name.split('_')[0].strip('Month')), name.split('Month')[1][2:].strip('BSL_')) if 'Month' in name else (0, name.strip('BSL_')) for name in df.columns], names= ['month', 'marker'])
df = df.stack('month')

display(df.head())
df.loc[:, ['AC1_Vit2', 'Matching IDs', 'Matching_VitorAC']] = df.loc[:, ['AC1_Vit2','Matching IDs', 'Matching_VitorAC']].groupby(['Tube_Label', 'Site','Collection_Date']).ffill() # for things common across samples
print(sorted(df.columns.tolist()))
print('num rows:', len(df))
display(df.isna().sum()) # any nan values?

feature_columns = [col for col in df.columns if col not in ['AC1_Vit2','Matching IDs', 'Matching_VitorAC']]
idx = pd.IndexSlice



df_background = df.loc[(df.index.get_level_values('Tube_Label')=='BACKGROUND'), :]
df = df.loc[~(df.index.get_level_values('Tube_Label')=='BACKGROUND'), :]
df.loc[:, 'patient']=[int(re.search(r"\d+", x).group(0)) for x in df.index.get_level_values('Tube_Label')]
df.loc[:, ['AC1_Vit2', 'Matching IDs', 'Matching_VitorAC']] = df.loc[:, ['AC1_Vit2', 'Matching IDs', 'Matching_VitorAC']].apply(pd.to_numeric)
df=df.reset_index().set_index(['patient', 'AC1_Vit2','month'])

df_orig = df.copy()

<h2> Sample Degradation Check </h2>

Samples were tested after a freeze-thaw cycle near collection, after 3 months, and after 9 months. All samples are normalised by subtracting the week-0 mean and dividing by the week-0 standard deviation. PCA is completed to see the proximity of samples in a higher dimensional space after degradation.

In [ ]:
import sklearn.decomposition
import sklearn.manifold
from matplotlib.pyplot import cm
import pylab

arrowsize=0.3

df=df_orig.copy()

index = pd.MultiIndex.from_product([set(df.index.get_level_values('patient')), (1,2), (0,3,9)], names=['patient', 'AC1_Vit2','month'])
df = df.reindex(index)


# normalise the data
mean_ = df.loc[idx[:, :, 0], feature_columns].mean()
std_ = df.loc[idx[:, :, 0], feature_columns].std()

try:df.loc[:, feature_columns] = (df.loc[:, feature_columns].values - mean_.values[np.newaxis, :])/std_.values[np.newaxis, :]
except TypeError:
    for col in feature_columns:
        print(col, df[col].dtype)

display(df.head())


pca = sklearn.decomposition.PCA(n_components=2,)
# pca.fit(df.loc[idx[:, :, 0], feature_columns+['AC1_Vit2']].values)
pca.fit(df[feature_columns].dropna(how='any').loc[idx[:, :, 0], feature_columns].values)

print('Explained variance:', pca.explained_variance_)
print('Principal components')
display(pd.DataFrame(pca.components_, columns = feature_columns))

# # Uncomment for list offeature influences on the principle axis
# s = pd.DataFrame(pca.components_, columns = feature_columns).loc[0, :]
# print(sorted(list(zip(s.index, s)), key=lambda x:-x[1]))

# # for other axis
# s = pd.DataFrame(pca.components_, columns = feature_columns).loc[1, :]
# print(sorted(list(zip(s.index, s)), key=lambda x:-x[1]))

one = pca.transform(df.loc[idx[:, :, 0], feature_columns].fillna(-1).values)
three = pca.transform(df.loc[idx[:, :, 3], feature_columns].fillna(-1).values)
nine = pca.transform(df.loc[idx[:, :, 9], feature_columns].fillna(-1).values)



background_one = pca.transform((df_background.loc[idx[:, :,:, 0], feature_columns].values-mean_[np.newaxis,:])/std_[np.newaxis,:])
background_three = pca.transform((df_background.loc[idx[:, :,:, 3], feature_columns].values-mean_[np.newaxis,:])/std_[np.newaxis,:])
background_nine = pca.transform((df_background.loc[idx[:, :,:, 9], feature_columns].values-mean_[np.newaxis,:])/std_[np.newaxis,:])

colours = iter(cm.PuOr(np.linspace(0, 1, len(one))))
# print(one.shape, three.shape, nine.shape)

colours = ['#785ef0', '#fe6100']
fig = plt.figure(figsize=(6,6))

pt_connections={}
for row in range(len(df.loc[idx[:, :, 0], :])):
    
    if any(df.loc[idx[:, :, 0], :].iloc[row].isna()):
        continue
        
    
    # colour by AC and vit
    assert 'AC1_Vit2'==df.index.names[1]        
    pt, ac_vit, _ = (df.loc[idx[:, :, 0], :].iloc[row]).name
    if ac_vit==1:
        pt_connections.update({pt:(one[row, 0], one[row,1])})
    c=colours[int(ac_vit-1)]
    

    plt.arrow(one[row, 0], one[row,1], three[row,0]-one[row, 0], three[row,1]-one[row, 1], color=c, head_width=arrowsize, head_length=arrowsize, head_starts_at_zero=False)

    if df.loc[idx[:, :, 9], :].iloc[row].isna().sum()==0:
        plt.arrow(three[row, 0], three[row,1], nine[row,0]-three[row, 0], nine[row,1]-three[row, 1], color=c, head_width=arrowsize, head_length=arrowsize, head_starts_at_zero=False)
    if (ac_vit==2)&(pt in pt_connections.keys()):
        # plot the dotted connector
        plt.plot([one[row,0], pt_connections[pt][0]],[one[row,1], pt_connections[pt][1]], linestyle='--', c='k', alpha=0.3)
    

plt.arrow(background_one[0, 0], background_one[0,1], background_three[0,0]-background_one[0,0], background_three[0,1]-background_one[0,1], color='black', head_width=arrowsize, head_length=arrowsize, head_starts_at_zero=False)
plt.arrow(background_three[0, 0], background_three[0,1], background_nine[0,0]-background_three[0,0], background_nine[0,1]-background_three[0,1], color='black', head_width=arrowsize, head_length=arrowsize, head_starts_at_zero=False)

plt.savefig('PCA_AC_Vit.png')

plt.title('PCA of samples at 0, 3, and 9 months')

plt.show()


fig = pylab.figure()
figlegend = pylab.figure(figsize=(3,2))
ax = fig.add_subplot(111)
lines = ax.scatter(range(10), pylab.randn(10), color = colours[0], label='AC')
lines = ax.scatter(range(10), pylab.randn(10),color = colours[1], label='Vit')
lines = ax.scatter(range(10), pylab.randn(10),color = 'k', label='Background')

handles,labels = ax.get_legend_handles_labels()
# figlegend.legend(lines, ('AC', 'Vit', 'Background'), 'center')
figlegend.legend(handles, labels, 'center')

# figlegend.show()
figlegend.savefig('legend.png')

<h2>Sample Location Preservation</h2>

In [ ]:
import scipy.stats as stats


display(df)
df = df_orig.copy()
# get a dense index

index = pd.MultiIndex.from_product([set(df.index.get_level_values('patient')), (1,2), (0,3,9)], names=['patient', 'AC1_Vit2','month'])

df = df.reindex(index)

display(df)

display(df_background.head())

print('Null hypothesis: medians are the same between AC and Vit at 0 months')
test_samples = df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 1,0], feature_columns].values - df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 2,0], feature_columns].values
# now eliminate rows with nans
test_samples = test_samples[~np.isnan(test_samples).any(axis=1), :]
# now do a 1d test on each axis independantly to see if there is a significant difference in values

results = []
for i, col in enumerate(feature_columns):
    s,p = stats.wilcoxon(test_samples[:, i].ravel())
    
    
    results.append((col, p, p<=0.05/len(feature_columns)/6,  test_samples[:, i].mean()))
    
display(pd.DataFrame(sorted(results, key=lambda x:x[1]), columns = ['marker','p-value','reject?', 'effect size']).set_index('marker'))
    
    
# repeat at 3 and 9 months
print('Null hypothesis: medians are the same between AC and Vit at 3 months')
test_samples = df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 1,3], feature_columns].values - df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 2,3], feature_columns].values
# now eliminate rows with nans
test_samples = test_samples[~np.isnan(test_samples).any(axis=1), :]
# now do a 1d test on each axis independantly to see if there is a significant difference in values

results = []
for i, col in enumerate(feature_columns):
    s,p = stats.wilcoxon(test_samples[:, i].ravel())
    
    
    results.append((col, p, p<=0.05/len(feature_columns)/6,  test_samples[:, i].mean())) #bonferonni correction accounted for by /len(feature_columns)/6
    
display(pd.DataFrame(sorted(results, key=lambda x:x[1]), columns = ['marker','p-value','refect?', 'effect size']).set_index('marker'))
    


    
# Is it the same across sample location?
print('Null hypothesis: medians are the same for AC between 0 and 3 months')

background_value = (df_background.loc[idx[:, :, :, 0], feature_columns] - df_background.loc[idx[:, :, :, 3], feature_columns].values) / df_background.loc[idx[:, :, :, 0], feature_columns].values

display(background_value)

test_samples = df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 1,0], feature_columns].values - df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 1,3], feature_columns].values
# now eliminate rows with nans
test_samples = test_samples[~np.isnan(test_samples).any(axis=1), :]
# now do a 1d test on each axis independantly to see if there is a significant difference in values

results = []
for i, col in enumerate(feature_columns):
    s,p = stats.wilcoxon(test_samples[:, i].ravel())
    
    
    results.append((col, p, p<=0.05/len(feature_columns)/6,  test_samples[:, i].mean(), test_samples[:, i].mean()/df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 1,0],col].mean(), background_value[col].values[0])) #bonferonni correction accounted for by /len(feature_columns)/6


display(pd.DataFrame(sorted(results, key=lambda x:x[1]), columns = ['marker','p-value','reject?', 'effect size', 'fraction_change', 'background_change']).set_index('marker'))
    
    
# Is it the same across sample location?
print('Null hypothesis: medians are the same for Vit between 0 and 3 months')
test_samples = df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 2,0], feature_columns].values - df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 2,3], feature_columns].values
# now eliminate rows with nans
test_samples = test_samples[~np.isnan(test_samples).any(axis=1), :]
# now do a 1d test on each axis independantly to see if there is a significant difference in values

results = []
for i, col in enumerate(feature_columns):
    s,p = stats.wilcoxon(test_samples[:, i].ravel())
    
    
    results.append((col, p, p<=0.05/len(feature_columns)/6,  test_samples[:, i].mean(), test_samples[:, i].mean()/df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), 1,0],col].mean(), background_value[col].values[0])) #bonferonni correction accounted for by /len(feature_columns)/6
    
display(pd.DataFrame(sorted(results, key=lambda x:x[1]), columns = ['marker','p-value','reject?', 'effect size', 'fraction_change', 'background_change']).set_index('marker'))
  

    
# Is it the same across sample location?
print('Null hypothesis: medians are the same between 0 and 3 months')
test_samples = df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), :,0], feature_columns].values - df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), :,3], feature_columns].values
# now eliminate rows with nans
test_samples = test_samples[~np.isnan(test_samples).any(axis=1), :]
# now do a 1d test on each axis independantly to see if there is a significant difference in values

results = []
for i, col in enumerate(feature_columns):
    s,p = stats.wilcoxon(test_samples[:, i].ravel())
    
    
    results.append((col, p, p<=0.05/len(feature_columns)/6,  test_samples[:, i].mean(), test_samples[:, i].mean()/df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), :,0],col].mean(), background_value[col].values[0])) #bonferonni correction accounted for by /len(feature_columns)/6
    
display(pd.DataFrame(sorted(results, key=lambda x:x[1]), columns = ['marker','p-value','reject?', 'effect size', 'fraction_change', 'background_change']).set_index('marker'))
  

    
    
# Is it the same across sample location?
print('Null hypothesis: medians are the same between 0 and 9 months')
test_samples = df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), :,0], feature_columns].values - df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), :,9], feature_columns].values
# now eliminate rows with nans
test_samples = test_samples[~np.isnan(test_samples).any(axis=1), :]
# now do a 1d test on each axis independantly to see if there is a significant difference in values

results = []
for i, col in enumerate(feature_columns):
    s,p = stats.wilcoxon(test_samples[:, i].ravel())
    
    
    results.append((col, p, p<=0.05/len(feature_columns)/6,  test_samples[:, i].mean(), test_samples[:, i].mean()/df.loc[idx[sorted(list(set(set(df.index.get_level_values('patient'))))), :,0],col].mean(), background_value[col].values[0])) #bonferonni correction accounted for by /len(feature_columns)/6
    
display(pd.DataFrame(sorted(results, key=lambda x:x[1]), columns = ['marker','p-value','reject?', 'effect size', 'fraction_change', 'background_change']).set_index('marker'))
  




<h2>ANOVA</h2>



In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#perform two-way ANOVA

#Matching_VitorAC

df_prime=df_orig.copy()

# df_prime.append(df_background)

df_prime.columns = [col.replace('_','').replace('(','').replace(')','').replace(' ','').replace('-','') for col in df_prime.columns]
display(df_prime.head())

end_df = []

for col in [c for c in feature_columns]:
    print('*'*10)
    print(col)
    # col ~ C(month) + C(AC1_Vit2) + C(month):C(AC1_Vit2)
    model = ols(f"{col.replace('_','').replace('(','').replace(')','').replace(' ','').replace('-','')} ~ C(month) + C(AC1_Vit2) + C(month):C(AC1_Vit2)", data=df_prime.reset_index()).fit()

    res = sm.stats.anova_lm(model, typ=2)
    print(res)
    col_=col.replace('_','').replace('(','').replace(')','').replace(' ','').replace('-','')
    print(df_prime[col_].groupby(['AC1_Vit2', 'month']).agg(['mean','std']))
        
    

    months = df_prime[col_].groupby('month').mean()
    ac1_vit2 = df_prime[col_].groupby('AC1_Vit2').mean()
    tmp_df = pd.DataFrame([[res.loc['C(month)', 'PR(>F)'], \
                           (months[3]-months[0])/months[0], (months[9]-months[3])/months[3], (months[9]-months[0])/months[0],\
                           res.loc['C(AC1_Vit2)', 'PR(>F)'],
                            (ac1_vit2[2]-ac1_vit2[1])/ac1_vit2[1], \
                            res.loc['C(month):C(AC1_Vit2)', 'PR(>F)']]],
                          columns = pd.MultiIndex.from_tuples([('Month', 'P-value'),('Month', 'Effect 0-3'),('Month', 'Effect 3-9'),('Month', 'Effect 0-9')
                                                                ,('Location', 'P-value'),('Location', 'Effect Vit:AC'),('Location-Month Interaction', 'P-value') ]),\
                          index=[col])
    end_df.append(tmp_df)
    
end_df = pd.concat(end_df)

display(end_df)

# bonferroni correction
def to_string(x, stat=True):
    if (abs(x)<0.001)&stat:
        return '<0.001'
    if np.isnan(x):
        return ''
    return '{:.3f}'.format(x)

for col in ['Effect 0-3', 'Effect 3-9', 'Effect 0-9']:
    end_df.loc[end_df[idx['Month', 'P-value']]>(0.05/len(feature_columns)), idx['Month', col]]=np.nan
    
end_df.loc[end_df[idx['Location', 'P-value']]>(0.05/len(feature_columns)), idx['Location', 'Effect Vit:AC']]=np.nan


end_df.loc[:, idx[:, 'P-value']] = end_df.loc[:, idx[:, 'P-value']].apply(lambda x: [to_string(i) for i in x] ) #*len(feature_columns)
end_df.loc[:, [col for col in end_df.columns.tolist() if 'Effect' in col[1]]] = end_df.loc[:, [col for col in end_df.columns.tolist() if 'Effect' in col[1]]].apply(lambda x: [to_string(i, stat=False) for i in x] ) #*len(feature_columns)


display(end_df)

end_df.to_csv('ANOVA_biomarker.csv')


    
    
    
"""
columns
marker name, month(p-value (sorted), reject null?, effect_size 0-3, effect_size 3-9, effect_size 0-9), location(p-value, effect_size), month&location(p-value, effect_size), background effect size
"""


<h2> What are the feature correlations?</h2>

a hierarchical clustering of feature covariances

In [ ]:
import seaborn as sns; #sns.set_theme(color_codes=True)


display(df_orig.head(10))
df_orig.sort_index(inplace=True)

df = df_orig.copy()
index = pd.MultiIndex.from_product([set(df.index.get_level_values('patient')), (1,2), (0,3,9)], names=['patient', 'AC1_Vit2','month'])
df = df.reindex(index)
display(df.head())
print(np.mean(df.loc[idx[:, :, 0], :].index.get_level_values('AC1_Vit2')))

display(df.loc[idx[:, :, 0], :].loc[df.loc[idx[:, :, 0], :]['Site'].notna(), feature_columns].groupby('patient').last().head(10))


fig = plt.figure()
g = sns.clustermap(df.loc[idx[:, :, 0], :].loc[df.loc[idx[:, :, 0], :]['Site'].notna(), feature_columns].rename(columns={item:item.split('(')[0] for item in df.columns.tolist()}).groupby('patient').last().corr())
plt.title('Feature Correlations Per Patient (Vit if available)')
plt.savefig('All_feature_correlations.png')
plt.show()



# vit only

display(df.loc[idx[:, :, 0], :].loc[df.loc[idx[:, :, 0], :]['Site'].notna(), feature_columns].head())

display()

fig = plt.figure()
g = sns.clustermap(df.loc[idx[:, 2, 0], :].loc[df.loc[idx[:, 2, 0], :]['Site'].notna(), feature_columns].rename(columns={item:item.split('(')[0] for item in df.columns.tolist()}).corr())
plt.title('Feature Correlations for Vit Samples')
plt.savefig('Vit_feature_correlations.png')
plt.show()



# ac_only

display()

fig = plt.figure()
g = sns.clustermap(df.loc[idx[:, 1, 0], :].loc[df.loc[idx[:, 1, 0], :]['Site'].notna(), feature_columns].rename(columns={item:item.split('(')[0] for item in df.columns.tolist()}).corr())
plt.title('Feature Correlations for AC Samples')
plt.savefig('AC_feature_correlations.png')
plt.show()



In [ ]:
from matplotlib.pyplot import cm

df = df_orig.copy()


index = pd.MultiIndex.from_product([set(df.index.get_level_values('patient')), (1,2), (0,3,9)], names=['patient', 'AC1_Vit2','month'])
df = df.reindex(index)

display(df.head())

for f in feature_columns:
    fig = plt.figure(figsize=(3, 6), dpi=300)
    plt.title(f)
    colours = iter(cm.viridis(np.linspace(0, 1, len(set(df.index.get_level_values('patient'))))))
    for patient in set(df.index.get_level_values('patient')):
        c=next(colours)
        vit = df.loc[idx[patient, 2],f].values
        plt.plot([0,3,9], vit, c=c, marker='x')
        
        ac = df.loc[idx[patient,1],f].values
        plt.plot([0,3,9], ac, c=c, marker='.')
        
    plt.plot([0,3,9], df_background[f].values, c='black', linestyle='--')
    
    plt.semilogy()
    plt.ylim([1, 50000])
    plt.savefig(f+'.png')
    
    new_xticks=[str(i)  if i in [0,3,9] else '' for i in range(10)]
    locs = list(range(10))
    plt.xticks(locs,new_xticks)
    plt.xlabel('Time (Months)')
    plt.savefig(f+'_ac_vit_coloured.png')
        
    plt.show()
    
    

from matplotlib.pyplot import cm

for f in feature_columns:
    fig = plt.figure(figsize=(3, 6), dpi=300)
    plt.title(f.split('(')[0])
    colours = iter(cm.viridis(np.linspace(0, 1, len(set(df.index.get_level_values('patient'))))))
    colours = ['#785ef0', '#fe6100']
    for patient in set(df.index.get_level_values('patient')):
        vit = df.loc[idx[patient, 2],f].values
        plt.plot([0,3,9], vit, c=colours[0], marker='x', alpha=0.7, label='Vit')
        
        ac = df.loc[idx[patient,1],f].values
        plt.plot([0,3,9], ac, c=colours[1], marker='.', alpha=0.7, label='AC')
        
    plt.plot([0,3,9], df_background[f].values, c='black', linestyle='--', label='Background')
    
    plt.semilogy()
    plt.ylim([1, 50000])
    plt.savefig(f+'_ac_vit_coloured.png')
    
    new_xticks=[str(i)  if i in [0,3,9] else '' for i in range(10)]
    locs = list(range(10))
    plt.xticks(locs,new_xticks)
    plt.xlabel('Time (Months)')
    plt.savefig(f+'_ac_vit_coloured.png')
        
    plt.show()
    